In [ ]:
!pip install -qU crewai[tools,agentops]==0.95.0 tavily-python scrapegraph-py

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai.tools import tool
import agentops
import os
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client
import json

In [ ]:
os.environ["AGENTOPS_API_KEY"] = ""
os.environ["COHERE_API_KEY"] = ""
os.environ["tvly_API_KEY"] = ""
os.environ["scrape_API_KEY"] = ""

In [ ]:
agentops.init(
    api_key = os.environ["AGENTOPS_API_KEY"],
    skip_auto_end_session = True
)

In [ ]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
llm = LLM(
    model="cohere/command-r-03-2024",
    temperature=0.0,
    provider="cohere"
)

search_client = TavilyClient(api_key = os.environ["tvly_API_KEY"])

scrape_client = Client(api_key = os.environ["scrape_API_KEY"])

In [ ]:
number_of_queries = 2

#Agent A

In [ ]:
class SuggestedSearchQueries(BaseModel):

    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items = 1, max_items = number_of_queries)

search_queries_recommendation_agent = Agent(
    role = "Search Queries Recommendation Agent", # a simple summary for the agent role
    goal = "\n".join([ # when to stop
        "To provide a list of suggested search queries based on a given search query.",
        "The queries must be varied and looking for specific items"
    ]),
    # backstory is the target of this agent
    backstory = "The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on context provided.",
    verbose = True,
    llm = llm
)

search_queries_recommndation_task = Task(
    description = "\n".join([
        "We are looking to buy a {product_name}",
        "Generate a list of {number_of_queries} search queries to find products on the following websites: {website_list}. ",
        "The search queries should be tailored to identify products that are relevant to our business and can be legally sold in {country}. ",
        "Ensure that the queries are specific enough to yield accurate results and cover a variety of product categories. ",
        "After generating the queries, verify if the products found can be sold in {country} based on local regulations and restrictions.",
    ]),
    expected_output = "A JSON object containing a list of suggested search queries.",

    output_json=SuggestedSearchQueries,

    output_file = os.path.join(output_dir, "step_1_suggested_search_queries.json"),

    agent = search_queries_recommendation_agent

    )

#Agent B

In [ ]:
class SingleSearchResult(BaseModel):

    title : str
    url : str
    content : str
    score : float
    search_query : str

class AllSearchResults(BaseModel):

    results: List[SingleSearchResult]

@tool
def search_engine_tool(query: str):
    """
    Performs a web search for a single query using a search engine. Useful for finding current information, products, or deals.

    Parameters:
    -----------
    query : str
        A single search query (e.g., "egypt coffee machine online shopping").

    Usage:
    ------
    Call the tool separately for each query:
    ```
    search_engine_tool(query="query 1")
    search_engine_tool(query="query 2")
    ```

    Returns:
    --------
    list or dict
        Search results (e.g., titles, URLs, snippets) from `search_client.search()`.

    Example:
    --------
    ```
    search_engine_tool(query="latest iPhone deals in Egypt")
    ```
    """
    return search_client.search(query)

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    verbose = True,
    llm = llm,
    tools = [search_engine_tool]
)

search_engine_task = Task(
    description = "\n".join([
        "The task is to seaech for products based on suggested search queries.",
        "You should collect results from multiple search queries.",
        "Ignore any susbitions links or not an ecommerce single product website link.",
        "Ignore any results with confidence score less than ({score}).",
        "The search result will be used to compare products prices from different websites."
        ]),
    expected_output = "A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file = os.path.join(output_dir, "step_2_search_results.json"),
    agent = search_engine_agent
)

#Agent C

In [ ]:
class ProductSpecs(BaseModel):

    specifications_name: str = Field(..., title = "the name of the specification")
    specifications_value: str = Field(..., title = "the value of the specification")


class SingleExtractedProduct(BaseModel):

    page_url: str = Field(..., title = "the original url for the product page")
    product_title: str = Field(..., title = "the title of the product")
    product_url: str = Field(..., title = "the url of the product")
    product_image_url: str = Field(..., title = "the url of the product image")
    product_current_price: float = Field(..., title = "the price of the product")
    product_original_price: float = Field(None, title = "the original price of the product")
    product_discount_percentage: float = Field(None, title = "the discount percentage of the product")

    product_specs: list[ProductSpecs] = Field(..., title = "the specifications of the product, focus on the most important specifications")

    agent_recommendation_rank: int = Field(..., title = "the rank of the product to be concederd in the final procurement report. (out of 5, the higher is better)")
    agent_recommendation_notes: list[str] = Field(..., title = "why you would recommend or not this product, compared to the other products.")

class AllExtractedProducts(BaseModel):

    products: List[SingleExtractedProduct]

@tool
def web_scraping_tool(page_url: str, required_fields: list):
    """
    Scrapes a web page to extract specific fields using a smart scraping client.
    """
    response = scrape_client.smartscraper(
        website_url = page_url,
        user_prompt = "Extract ```json\n" + SingleExtractedProduct.schema_json() + "```\nfrom the web page"
    )

    return response

scraping_agent = Agent(
    role = "Scraping agent",
    goal = "To extract some information from web pages.",
    backstory = "The agent is designed to extract information from web pages.",
    llm = llm,
    tools = [web_scraping_tool],
    verbose = True,
)

scraping_task = Task(
    description = "\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages url.",
        "collect the best {top_recommendation_number} products based on agent_recommendation_rank.",
        ]),
    expected_output = "A JSON object containing product details.",
    output_json = AllExtractedProducts,
    output_file = os.path.join(output_dir, "step_3_search_results.json"),
    agent = scraping_agent
)

#Agent D

In [ ]:
procurement_report_author_agent = Agent(
    role = "Procurement Report Author Agent",
    goal = "To generate an HTML procurement report based on the extracted product details.",
    backstory = "The agent is designed to generate an HTML procurement report based on the extracted product details.",
    verbose = True,
    llm = llm
)

procurement_report_author_task = Task(
    description = "\n".join(["the task is to generate an HTML procurement report based on the extracted product details.",
                             "you have to use Bootstrap CSS framwork for a better UI.",
                             "use the provided string_source is needed."
                             "the report should be constructed with the following sections:",
                             "Product Overview: Short descriptions of each product, including brand, model, and key features.",
                             "Comparison Criteria: Metrics used (e.g., price, performance, features) and their relevance.",
                             "Pros and Cons: Bullet-point list of strengths and weaknesses for each product.",
                             "Key Findings: Summary of how each product performs against the criteria.",
                             "Recommendation: Final suggestion on the best product for specific needs or budgets",
                             ]),
    expected_output = "A professional HTML page for the procurement report.",
    output_file = os.path.join(output_dir, "step_4_procurement_report.html"),
    agent = procurement_report_author_agent
)

In [ ]:
content = "Our company provides AI solutions to help companies to enhance their search quality."
string_source = StringKnowledgeSource(
    content = content,
)

In [ ]:
crew = Crew(
    agents = [
        search_queries_recommendation_agent,
        search_engine_agent,
        scraping_agent,
        procurement_report_author_agent
    ],

    tasks = [
        search_queries_recommndation_task,
        search_engine_task,
        scraping_task,
        procurement_report_author_task
    ],

    process = Process.sequential,
    knowledge_sources=[string_source]
)

In [ ]:
crew_result = crew.kickoff(
    inputs = {
        "product_name" : "laptop",
        "number_of_queries" : number_of_queries,
        "website_list" : ["amazon.com", "jumia.com"],
        "country" : "Egypt",
        "score" : 0.1,
        "top_recommendation_number" : 2
     }
)

# Agent: Search Queries Recommendation Agent
## Task: We are looking to buy a laptop
Generate a list of 2 search queries to find products on the following websites: ['amazon.com', 'jumia.com']. 
The search queries should be tailored to identify products that are relevant to our business and can be legally sold in Egypt. 
Ensure that the queries are specific enough to yield accurate results and cover a variety of product categories. 
After generating the queries, verify if the products found can be sold in Egypt based on local regulations and restrictions.


# Agent: Search Queries Recommendation Agent
## Final Answer: 
```json
{
  "queries": [
    "laptop with i7 processor amazon.com",
    "15 inch laptop jumia.com"
  ]
}
```


# Agent: Search Engine Agent
## Task: The task is to seaech for products based on suggested search queries.
You should collect results from multiple search queries.
Ignore any susbitions links or not an ecommerce single product website link.
Ignore any results w